In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression;
from sklearn.neighbors import KNeighborsClassifier;
from sklearn.tree import DecisionTreeClassifier;
from sklearn.linear_model import LogisticRegression;
from sklearn.ensemble import RandomForestClassifier;
from sklearn.metrics import confusion_matrix;
from sklearn import preprocessing;
from sklearn import metrics;

## Data Preprocessing

In [3]:
raw_data = pd.read_csv("train.csv");
raw_test = pd.read_csv("test.csv");

raw_data.groupby('Survived').mean()
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#plt.hist(raw_data['Age'], bins=100);
#plt.hist(raw_data['Fare'], bins=100);
#plt.scatter(raw_data['Age'], raw_data['Survived']);
raw_data['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [5]:
raw_data['Sex'] = (raw_data['Sex'] == "male")*1;
raw_data['Age'].replace(np.nan, raw_data['Age'].mean(), inplace=True);
raw_data['Age_cat'] = (raw_data['Age'] > 50)*1;
raw_data[['A','B','C','D','E','F','G','T']] = pd.get_dummies(raw_data['Cabin'].str[0:1]);
raw_data[['Embarked_S','Embarked_C','Embarked_Q']] = pd.get_dummies(raw_data['Embarked'].str[0:1]);


raw_test['Sex'] = (raw_test['Sex'] == "male")*1;
raw_test['Age'].replace(np.nan, raw_test['Age'].mean(), inplace=True);
raw_test['Fare'].replace(np.nan, raw_test['Fare'].mean(), inplace=True);
raw_test['Age_cat'] = (raw_test['Age'] > 50)*1;
raw_test[['A','B','C','D','E','F','G']] = pd.get_dummies(raw_test['Cabin'].str[0:1]);
raw_test[['Embarked_S','Embarked_C','Embarked_Q']] = pd.get_dummies(raw_test['Embarked'].str[0:1]);

raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,B,C,D,E,F,G,T,Embarked_S,Embarked_C,Embarked_Q
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,...,0,0,0,0,0,0,0,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,...,0,1,0,0,0,0,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,...,0,0,0,0,0,0,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,...,0,1,0,0,0,0,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,...,0,0,0,0,0,0,0,0,0,1


In [6]:
X = raw_data[['Pclass','Sex','Age','SibSp','Parch','Fare','A','B','C','D','E','F','G','Embarked_S','Embarked_C','Embarked_Q']];
# X = raw_data[['Pclass','Sex']];
Y = raw_data['Survived'];

X_test = raw_test[['Pclass','Sex','Age','SibSp','Parch','Fare','A','B','C','D','E','F','G','Embarked_S','Embarked_C','Embarked_Q']];
# X_test = raw_data[['Pclass','Sex']];

#X = preprocessing.StandardScaler().fit(X).transform(X);
#X_test = preprocessing.StandardScaler().fit(X_test).transform(X_test);

X = pd.DataFrame(X);
X_test = pd.DataFrame(X_test);

#X[1]= raw_data['Sex'];
#X_test[1] = raw_test['Sex'];
X

,Pclass,Sex,Age,SibSp,Parch,Fare,A,B,C,D,E,F,G,Embarked_S,Embarked_C,Embarked_Q
0,3,1,22.000000,1,0,7.2500,0,0,0,0,0,0,0,0,0,1
1,1,0,38.000000,1,0,71.2833,0,0,1,0,0,0,0,1,0,0
2,3,0,26.000000,0,0,7.9250,0,0,0,0,0,0,0,0,0,1
3,1,0,35.000000,1,0,53.1000,0,0,1,0,0,0,0,0,0,1
4,3,1,35.000000,0,0,8.0500,0,0,0,0,0,0,0,0,0,1
5,3,1,29.699118,0,0,8.4583,0,0,0,0,0,0,0,0,1,0
6,1,1,54.000000,0,0,51.8625,0,0,0,0,1,0,0,0,0,1
7,3,1,2.000000,3,1,21.0750,0,0,0,0,0,0,0,0,0,1
8,3,0,27.000000,0,2,11.1333,0,0,0,0,0,0,0,0,0,1
9,2,0,14.000000,1,0,30.0708,0,0,0,0,0,0,0,1,0,0


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split( X, Y, test_size=0.3) #, random_state=5)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_val.shape,  y_val.shape)

Train set: (623, 16) (623,)
Test set: (268, 16) (268,)


## K Neigh Classification

In [205]:
neigh = KNeighborsClassifier(n_neighbors = 6).fit(X_train,y_train)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Train set Accuracy: ", metrics.accuracy_score(y_val, neigh.predict(X_val)))

Train set Accuracy:  0.7656500802568218
Train set Accuracy:  0.6902985074626866


## Decision Tree Classification

In [40]:
decisionTree = DecisionTreeClassifier(criterion="entropy", max_depth = 20);
decisionTree.fit(X_train, y_train);
print("Train set Accuracy: ", metrics.accuracy_score(y_train, decisionTree.predict(X_train)))
print("Validation set Accuracy: ", metrics.accuracy_score(y_val, decisionTree.predict(X_val)))



decision_tree = DecisionTreeClassifier();
decision_tree.fit(X_train, y_train);
Y_pred = decision_tree.predict(X_val);

print(round(decision_tree.score(X_train, y_train) * 100, 2))
print(round(decision_tree.score(X_val, y_val) * 100, 2))

Train set Accuracy:  0.985553772070626
Validation set Accuracy:  0.7425373134328358
98.72
74.25


In [41]:
from sklearn.model_selection import cross_val_score
rf = DecisionTreeClassifier()
scores = cross_val_score(rf, X, Y, cv=20, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.76086957 0.80434783 0.68888889 0.8        0.73333333 0.75555556
 0.75555556 0.86666667 0.88888889 0.72727273 0.86363636 0.72727273
 0.72727273 0.88636364 0.79545455 0.75       0.84090909 0.81818182
 0.79545455 0.84090909]
Mean: 0.7913416776460256
Standard Deviation: 0.05776340739328497


## Random Forest

In [35]:
random_forest = RandomForestClassifier(n_estimators=100);
random_forest.fit(X_train, y_train);
Y_pred = random_forest.predict(X_train);
#print(round(random_forest.score(X_train, y_train) * 100, 2));
#print(round(random_forest.score(X_val, y_val) * 100, 2));

Scores: [0.73333333 0.77777778 0.75280899 0.86516854 0.87640449 0.83146067
 0.78651685 0.76404494 0.87640449 0.85227273]
Mean: 0.8116192827147882
Standard Deviation: 0.051898161053629314


In [45]:
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, X, Y, cv=10, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.75555556 0.78888889 0.76404494 0.87640449 0.86516854 0.82022472
 0.80898876 0.78651685 0.85393258 0.84090909]
Mean: 0.8160634434229941
Standard Deviation: 0.04018666988598526


## Logistic Regression Classification

In [211]:
LR = LogisticRegression(C=0.3, solver='liblinear').fit(X_train,y_train);
print("Train set Accuracy: ", metrics.accuracy_score(y_train, LR.predict(X_train)))
print("Validation set Accuracy: ", metrics.accuracy_score(y_val, LR.predict(X_val)))

Train set Accuracy:  0.7961476725521669
Validation set Accuracy:  0.8059701492537313


In [212]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,A,B,C,D,E,F,G,Embarked_S,Embarked_C,Embarked_Q
0,3,1,34.5,0,0,7.8292,0,0,0,0,0,0,0,0,1,0
1,3,0,47.0,1,0,7.0000,0,0,0,0,0,0,0,0,0,1
2,2,1,62.0,0,0,9.6875,0,0,0,0,0,0,0,0,1,0
3,3,1,27.0,0,0,8.6625,0,0,0,0,0,0,0,0,0,1
4,3,0,22.0,1,1,12.2875,0,0,0,0,0,0,0,0,0,1


In [213]:
final = raw_test[['PassengerId']];
final['Survived'] = decisionTree.predict(X_test)

C:\Users\dhruv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [214]:
final.to_csv('final.csv', index=False)

In [218]:
final = raw_test[['PassengerId','Sex']];
final['Survived'] = LR.predict(X_test);
final[final['Sex'] == 0]['Survived']

C:\Users\dhruv\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1      0
4      0
6      0
8      1
12     1
14     1
15     1
18     0
19     0
22     1
24     1
26     1
32     0
33     0
36     0
37     1
43     1
44     1
48     1
49     0
52     1
53     1
59     1
63     1
65     1
66     1
69     1
70     1
72     0
74     1
      ..
347    0
349    1
350    1
354    1
356    1
359    0
361    1
362    1
364    1
365    0
367    1
368    1
371    1
374    1
375    1
376    0
382    0
383    1
385    1
391    1
395    1
397    1
400    1
402    1
408    0
409    1
410    0
411    1
412    1
414    1
Name: Survived, Length: 152, dtype: int64